In [1]:
from common import *
from matrix import *
from helper.weighted import *
%matplotlib inline
canvas(16, 12)
import pickle
import gc

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [2]:
from models.thealg import TheAlg
from models.split import split3

In [3]:
o = O()
def_W(o); W = o.Wa.astype(np.float32)
def_Y(o); Y = o.Ya.astype(np.float32)

#### x features {

In [4]:
of = O()

In [5]:
def_X(o, include='logs z1 wqtl wnrm')
of.Xwnrm = o.Xwnrm
of.Xlogz1s = o.Xlogz1s.astype(np.float32)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)
/home/ec2-user/Notebooks/g-research-ffc2018/helper/weighted.ipynb:11: RuntimeWarning: invalid value encountered in true_divide
  "import notebook_import_hook\n",
/home/ec2-user/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1901: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1901: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)


In [6]:
of.bias = dh[['Count']].makesw().fillna(1).astype(np.float32)
of.bias.columns = pd.MultiIndex.from_product([['bias'], range(3023)], names=['Feature', 'Stock'])

In [7]:
of.xfd = dh[['Day']]
of.xfdw = ((of.xfd + 2) % 7).colname_append('.wkd')
of.Xfdt = pd.concat([of.xfd, of.xfdw], axis=1).pipe(makesw).pipe(o._dewmean).astype(np.float32)

In [8]:
save_dir = '/big/data/features/'
pathname = save_dir + '{}sw32/{}'
xabcs = dict(
    X3int = ('', 'x3.log.zc(-93).mintco0[4,1]'),
    Xgint = ('', 'xg.log.zc(-93).mintco0[6,1][3,2]'),
    #X3intwqtl = ('', 'x3.log.zc(-93).mintco0[4,1].wqtl'),
    #Xgintwqtl = ('', 'xg.log.zc(-93).mintco0[6,1][3,2].wqtl'),
    X3intwnrm = ('', 'x3.log.zc(-93).mintco0[4,1].wnrm'),
    Xgintwnrm = ('', 'xg.log.zc(-93).mintco0[6,1][3,2].wnrm'),
    Xowind = ('window/', 'xo.log.z(1).windiff'),
    X3wind = ('window/', 'x3.log.zc(-93).mintco0[2,1].windiff'),
    Xgwind = ('window/', 'xg.log.zc(-93).mintco0[2,1].windiff'),
    #Xowindwqtl = ('window/', 'xo.log.z(1).windiff.wqtl'),
    #X3windwqtl = ('window/', 'x3.log.zc(-93).mintco0[2,1].windiff.wqtl'),
    #Xgwindwqtl = ('window/', 'xg.log.zc(-93).mintco0[2,1].windiff.wqtl'),
    Xowindwnrm = ('window/', 'xo.log.z(1).windiff.wnrm'),
    X3windwnrm = ('window/', 'x3.log.zc(-93).mintco0[2,1].windiff.wnrm'),
    Xgwindwnrm = ('window/', 'xg.log.zc(-93).mintco0[2,1].windiff.wnrm'),
)
for xname, abc in xabcs.items():
    of[xname] = pd.read_hdf(pathname.format(*abc))
xabcs['Xwnrm'] = ('', 'xo.log.z(1).wnrm')
for xname in 'X3int Xgint Xowind Xgwind X3wind'.split():
    of[xname] = of[xname].pipe(o._dewmean)

optional {

In [11]:
makepos = lambda X: X.clip(lower=0).colname_append('.pos', level=0).pipe(o._dewmean).astype(np.float32)
makeneg = lambda X: X.clip(upper=0).colname_append('.neg', level=0).pipe(o._dewmean).astype(np.float32)

In [19]:
for x in 'Xwnrm X3intwnrm Xgintwnrm Xowindwnrm X3windwnrm Xgwindwnrm'.split():
    print(x)
    xp = of[x].pipe(makepos)
    xp.to_hdf(pathname.format(*xabcs[x]) + '.pos', key='a')
    xn = of[x].pipe(makeneg)
    xn.to_hdf(pathname.format(*xabcs[x]) + '.neg', key='a')
    del of[x]

X3intwnrm
Xgintwnrm
Xowindwnrm
X3windwnrm
Xgwindwnrm


} optional end

In [9]:
xraw = 'bias Xfdt Xlogz1s X3int Xgint Xowind X3wind Xgwind'.split()
xq = 'Xwnrm X3intwnrm Xgintwnrm Xowindwnrm X3windwnrm Xgwindwnrm'.split()
xpn = 'Xwnrm X3intwnrm Xgintwnrm Xowindwnrm X3windwnrm Xgwindwnrm'.split()

In [10]:
for x in xpn:
    of[x + 'p'] = pd.read_hdf(pathname.format(*xabcs[x]) + '.pos')
    of[x + 'n'] = pd.read_hdf(pathname.format(*xabcs[x]) + '.neg')

In [22]:
X = pd.concat([of[x] for x in xraw] + [of[x] for x in xq],# + [of[x + 'p'] for x in xpn] + [of[x + 'n'] for x in xpn],
              axis=1, copy=False).astype(np.float32, copy=False)

In [23]:
assert X.columns.levels[0].shape[0] * 3023 == X.columns.__len__()
assert X.columns.levels[1].shape[0] == 3023

In [ ]:
temp = '5zombie'
#X.to_hdf('/big/data/ready/algrun' + temp, key='X')
X = pd.read_hdf(top_dir + 'data/ready/algrun' + temp, key='X')

In [ ]:
mvalid = o.Y.count()

In [6]:
try:
    dict.clear(o)
    del o
except NameError:
    pass
gc.collect()
try:
    dict.clear(of)
    del of
except NameError:
    pass
gc.collect()

0

#### } x features end

In [7]:
sfilt = ~stock_nvdf(dh, 249) & (stockmarket != 3)

In [24]:
oW, oX, oY, omvalid = W, X, Y, mvalid

In [25]:
W = W.T[sfilt].T
Y = Y.T[sfilt].T
X = X.wide_filter(sfilt).chain.fillna(0, inplace=True)
mvalid = mvalid[sfilt]
gc.collect()

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


0

In [26]:
s = split3(X, repeat=2)
ta = TheAlg(X, Y, W, s, mvalid, algorithm='expfb')
ta._args = ([0, 4, 14, 32], [25, 50, 75, 100, 125, 150])

In [27]:
try:
    del X, Y, W
except NameError:
    pass
gc.collect()

60

In [28]:
assert ta.columns.get_level_values(0).nunique() * ta.columns.get_level_values(1).nunique() == ta.columns.shape[0]
assert ta.k * ta.m == ta.columns.shape[0]

In [30]:
branch_at_depth = lambda d: [15, 5, 3, 1][d] if d < 4 else 1
ta.alg1(branch_at_depth=branch_at_depth, max_depth=4, dot=10, numdot=10)

(0.7769072846197803, 0.8182800823947572) None 0
[] ()

<1285> 0.........100.........200.........300.........400.........500.........600.........700.........800.........900.........1000.........1100.........1200........

(0.6689772825848184, 0.705450450414193) ((0, 50), (14, 25)) 1
[xg.log.zc(-93).sgnint(03/44).wnrm] (455,)

<1284> 0.........100.........200.........300.........400.........500.........600.........700.........800.........900.........1000.........1100.........1200........

(0.6615748591502816, 0.7031368551196607) ((4, 50), (0, 50)) 2
[xg.log.zc(-93).sgnint(03/44).wnrm, xg.log.zc(-93).sgnint(35/44).wnrm.neg] (455, 1130)

<1283> 0.........100.........200.........300.........400.........500.........600.........700.........800.........900.........1000.........1100.........1200........

(0.656378396632536, 0.7010856135418398) ((4, 75), (0, 75)) 3
[xg.log.zc(-93).sgnint(03/44).wnrm, xg.log.zc(-93).sgnint(35/44).wnrm.neg, x3E.log.z(1).window(<20>*!-).wnrm] (455, 1130, 523)

<1282

KeyboardInterrupt: 

## diagnosis analysis

In [31]:
pname = '[~249&~m3]zombs(0,4,14,32)(25,50,75,100,125,150)'
featnames = [str(ta.columns[ta.m * f][0]) for f in range(ta.k)]

In [32]:
with open(top_dir + 'data/progress/linsearch/' + pname + '.pkl', 'wb') as file:
    pickle.dump((featnames, ta.scores, ta.fbnums), file, protocol=-1)

In [33]:
see = {tuple(featnames[j] for j in k): s for k, s in ta.scores.items() if k is not None}
tc0, ho0 = ta.scores[None]
sub = [(tc / tc0, ho / ho0, k) for k, (tc, ho) in see.items()
       if k is not None]
_ = [print(line) for line in sorted(sub)[:100]]

(0.652535685895395, 0.7004064900116118, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3E.log.z(1).window(<20>*!-).wnrm.pos', 'xg.log.zc(-93).sgnint(12/33).wnrm.neg'))
(0.6529830325397746, 0.6889513593543053, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'xg.log.zc(-93).sgnint(25/33).wnrm.neg', 'x3.log.zc(-93).sgnint(1/4).window(<20>*!-).wnrm.neg'))
(0.6530042363420493, 0.6926842767326568, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3.log.zc(-93).sgnint(0/4).window(<20>*!-).wnrm.neg', 'xg.log.zc(-93).sgnint(12/33).wnrm.neg'))
(0.6532938175117425, 0.6915756647607065, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3.log.zc(-93).sgnint(0/4).window(<20>*!-).wnrm.neg', 'xg.log.zc(-93).sgnint(03/12).wnrm.pos'))
(0.6534188969807563, 0.6888000588946109, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3.log.zc(-93).sgnint(0/4).window(<20>*!-).wnrm.neg', 'xg.log.zc(-93).sgnint(25/33).wnrm.neg'))
(0.6534188969807563, 0.6888000588946109, ('xg.log.zc(-93).sgnint(03/44).wnr